# Evaluate Summary Readability

This notebook loads a document from Firestore, extracts the summary, and evaluates its readability using the MES framework.

In [ ]:
import sys
import os
import logging
import pandas as pd

# Add src directory to path
sys.path.append('../src')

from orchestrator.experiment_runner import ExperimentRunner
from clients.firestore_client import FirestoreClient

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

## Load Data from Firestore

In [ ]:
# Configuration
PROJECT_ID = "gcp-project-id"  # Replace with your GCP project ID
COLLECTION_ID = "summaries"
DOCUMENT_ID = "004bd5df-fbb2-4c5a-8997-c317e18907f4"
VERSION_ID = "1751461391"

# Initialize Firestore client
firestore_client = FirestoreClient(project_id=PROJECT_ID)

# Load the document
try:
    document = firestore_client.get_document(f"{COLLECTION_ID}/{DOCUMENT_ID}/versions", VERSION_ID)
    summary_text = document.get("summary", {}).get("body")
    print("Successfully loaded summary.")
    print("---")
    print(summary_text)
    print("---")
except FileNotFoundError as e:
    print(e)
    summary_text = None

## Evaluate Readability

In [ ]:
if summary_text:
    # Initialise the experiment runner
    config_path = '../config/readability_experiment.yaml'
    runner = ExperimentRunner(config_path)

    # Prepare the response data
    response_data = {
        "response": summary_text,
        "model_id": "firestore-document" 
    }

    # Evaluate the response
    results_df = runner.evaluate_response("readability_check", response_data)

    # Display results
    print("Readability Scores:")
    display(results_df[['experiment_name', 'readability_flesch_reading_ease', 'readability_flesch_kincaid_grade', 'readability_gunning_fog']].head())
else:
    print("Summary text not available, skipping evaluation.")